In [9]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

<style>
div.blue{
    background-color:#e6f0ff; 
    border-radius: 5px; 
    padding: 20px;}
</style> 

<style>
div.warn {    
    background-color: #fcf2f2;
    border-color: #dFb5b4;
    border-left: 5px solid #dfb5b4;
    padding: 0.5em;
    }
 </style>
    
<h1 style="text-align: center; color: purple;" markdown="1">Econ 320 Python: Heteroscedasticity </h1>
<h2 style="text-align: center; color: purple;" markdown="1">Handout # 12 </h2>

 


#### Package setup

In [29]:
import wooldridge as woo
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels.api as sm
import patsy as pt

from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

# Heteroscedasticity

The homoscedasticity assumptions require that the variance of the error terms is unrelated to the regressors, i.e. 

$$Var(u|x_1, ..., x_k)=\sigma^2.$$ 

If homoscedasticity is violated, the standard errors are invalid and all inferences from t, F and other tests based on them are unreliable. In that case, we are facing heteroscedasticity in the standard errors: the circumstance in which the variability of the standard errors is unequal across the range of values of the predicted value of the dependent variable.

## Heteroscedasticity-Robust Inference


As you learned in class you need to modify the standard errors to correct for heteroscedasticity. This is having "heteroscedasticity-robust standard errors." 

In Python, we can do this using the `statsmodels` package. Using the  argument `cov_type` in the method `.fit()` we can obtain regression results that produced several refined versions of the white formula presented in your book. (Wooldridge, 2019). 

Let's say that the results from your regression are stored in the object *reg.*, then the variance- covariance matrix can be calculated using 

* **reg.fit(cov_type='nonrobust')** or **reg.fit()** for the default homoscedasticy-based standard errors. 
* **reg.fit(cov_type='HC0')**  for the classical version of White's robust varinace-covariance matrix by Wooldridge (2019, Equation 8.4 in Section 8,2)
* **reg.fit(cov_type='HC1')**  for the classical version of White's robust varinace-covariance matrix corrected by degrees of freedom.  
* **reg.fit(cov_type='HC2')**  for a version with small sample correction. This isndefault behavious of Stata. 
* **reg.fit(cov_type='HC3')**  for the refined version of version White's robust variance- covariance matrix. 

### Example with GPA data from wooldridge

For the spring semester run the regression 

$$ Cumulative GPA = \beta_0 + \beta_1SAT + \beta_2HSpercentile + \beta_3Totalhours + \beta_4 female + \beta_5 black + \beta_6 white $$


In [24]:
gpa3 = woo.dataWoo('gpa3')

# define regression model:
reg = smf.ols(formula='cumgpa ~ sat + hsperc + tothrs + female + black + white',
              data=gpa3, subset=(gpa3['spring'] == 1))

# estimate default model (only for spring data):
results_default = reg.fit()


In [25]:
# estimate model with White SE (only for spring data):
results_white = reg.fit(cov_type='HC0')

# estimate model with refined White SE (only for spring data):
results_refined = reg.fit(cov_type='HC3')



## Inference 
### T-test  another way of doing it

in this example we are going to test $H_0: \beta_4 =0 $ if the coefficient associated with female variables is statistically significant. We learned a few ways of doing this in the inference chapter and we also know that the resulst table gives us the answer. but here is another quick way. We are going to do the test for the results assuming homoskedasticity and assuming heteroskedasticity. ( Yes it can also be written heteroscedasticity)

Once you define the hypothesis you can use the function `t_test()` to test for the significance of your results see below and compare the results. The coefficnets remain the same, the standard errors change, but in this case we still reject the null hypothesis. 

In [26]:
H0 = 'female = 0'
t_test = results_default.t_test(H0) # Assuming Homoskedasticity
print(t_test)
t_test = results_refined.t_test(H0) # Assuming Heteroskedasticity
print(t_test)

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.3034      0.059      5.141      0.000       0.187       0.420
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.3034      0.060      5.054      0.000       0.186       0.421


In [27]:
# put all results in stargazer table and 
models = Stargazer([results_default, results_white, results_refined])
#models.covariate_order(['Intercept','female' , 'educ' , 'exper', 'tenure', 'married',])
HTML(models.render_html())

### The F- test 

For the joint significance of all the coefficients you can check results for the F test on the regression table or  stargazer table. 

For other joint hypothesis like 

$$H_0: \beta_5 = 0 \: \&  \: \beta_6=0$$

See examples below. 
 

In [31]:
# definition of model and hypotheses:
reg = smf.ols(formula='cumgpa ~ sat + hsperc + tothrs + female + black + white',
              data=gpa3, subset=(gpa3['spring'] == 1))
hypotheses = ['black = 0', 'white = 0']

# F-Tests using different variance-covariance formulas:
# ususal VCOV:
results_default = reg.fit()
ftest_default = results_default.f_test(hypotheses)
fstat_default = ftest_default.statistic[0][0]
fpval_default = ftest_default.pvalue
print(f'fstat_default: {fstat_default}\n')
print(f'fpval_default: {fpval_default}\n')

# refined White VCOV:
results_hc3 = reg.fit(cov_type='HC3')
ftest_hc3 = results_hc3.f_test(hypotheses)
fstat_hc3 = ftest_hc3.statistic[0][0]
fpval_hc3 = ftest_hc3.pvalue
print(f'fstat_hc3: {fstat_hc3}\n')
print(f'fpval_hc3: {fpval_hc3}\n')

# classical White VCOV:
results_hc0 = reg.fit(cov_type='HC0')
ftest_hc0 = results_hc0.f_test(hypotheses)
fstat_hc0 = ftest_hc0.statistic[0][0]
fpval_hc0 = ftest_hc0.pvalue
print(f'fstat_hc0: {fstat_hc0}\n')
print(f'fpval_hc0: {fpval_hc0}\n')

fstat_default: 0.6796041956073353

fpval_default: 0.5074683622584049

fstat_hc3: 0.6724692957656578

fpval_hc3: 0.5110883633440992

fstat_hc0: 0.7477969818036153

fpval_hc0: 0.4741442714738484



## Heteroskedasticity test

The null hypothesis for this tests is that the homoscesdasticity assumption is true. Meaning that the variance of the error term should be constant. $$ H_0: E(u|X)=\Phi^2$$

Breush-Pagan test (BP test) for heteroscedasticity is easy to implement with basic OLS routines. 

 1. Run the resgression $y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \beta_2x_2 + ... + \beta_kx_k$
 2. Obtain the residuals $\hat{u}$
 3. Regress $\hat{u}^2= \delta_0 + \delta_1x_1 + \delta_2x_2 + \delta_2x_2 + ... + \delta_kx_k = e$  
 4. Test for joint significance of all the independent variables. The F test. This will give you an LM statictic  that is aproximatelly distributed $\chi^2$
 
The LM version of the BP test is very convinient to use with the **statsmodels** function **stats.diagnostic.het_breuschpagan** It can be used directly. 

We can also use the *bptest* command to do the calculations of the LM version of the test.

In [32]:
hprice1 = woo.dataWoo('hprice1')

# estimate model:
results=smf.ols("price~lotsize+sqrft+bdrms", data=hprice1).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     57.46
Date:                Fri, 19 Nov 2021   Prob (F-statistic):           2.70e-20
Time:                        12:08:58   Log-Likelihood:                -482.88
No. Observations:                  88   AIC:                             973.8
Df Residuals:                      84   BIC:                             983.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -21.7703     29.475     -0.739      0.462     -80.385      36.844
lotsize        0.0021      0.001      3.220      0.002       0.001       0.003
sqrft          0.1228      0.013      9.275      0.000       0.096       0.149
bdrms         13.8525      9.010      1.537      0.128      -4.065      31.770
==============================================================================
Omnibus:                       20.398   Durbin-Watson:                   2.110
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.278
Skew:                           0.961   Prob(JB):                     9.79e-08
Kurtosis:                       5.261   Cond. No.                     6.41e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [34]:
# automatic BP test (LM version):

from statsmodels.compat import lzip
test = sms.het_breuschpagan(results.resid, results.model.exog)
name = ['Lagrange Multiplier Statistic', 'P-Value', 'F-Value', 'F P-Value']
pd.DataFrame(lzip(name, test))

,0,1
0,Lagrange Multiplier Statistic,14.092386
1,P-Value,0.002782
2,F-Value,5.338919
3,F P-Value,0.002048


In [35]:
# Manual regression of squared residuals 
smf.ols("np.power(results.resid,2) ~ lotsize+sqrft+bdrms", data=hprice1).fit().f_pvalue

0.0020477444209360787

#### The White test 

This test is a variant of the BP test where we regress the residuals on the fitted values of $\hat{y}$ and $\hat{y}^2$

The example below estimates a model of housing prices and tests for heteroscedasticity in the standard errors. In the two cases below, both tests do not reject the null hypotesis at conventionally significant levels. 
> Look at the p values. 

In [38]:
from statsmodels.compat import lzip

dato = woo.dataWoo('hprice1')
results=smf.ols("np.log(price)~np.log(lotsize)+np.log(sqrft)+bdrms", data=hprice1).fit()

# BP test
test = sms.het_breuschpagan(results.resid, results.model.exog)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print('B-P Test')
display(pd.DataFrame(lzip(labels,test)))


# White test standard version 1 
result_white_predictions = sms.het_white(results.resid, results.model.exog)
print('White Test (version 1)')
display(pd.DataFrame(zip(labels, result_white_predictions)))

# White test version 2 
# create the data with 
X_wh = pd.DataFrame({'const':1, 'fitted_reg':results.fittedvalues, 'fitted_reg_sq': results.fittedvalues**2})
result_white_predictions = sms.het_breuschpagan(results.resid, X_wh)
print('White Test (Version 2)')
# lzip or zip the output array into a datafram for a nice looking table.
display(pd.DataFrame(lzip(labels,result_white_predictions)))



B-P Test


,0,1
0,LM Statistic,4.223246
1,LM-Test p-value,0.238345
2,F-Statistic,1.411500
3,F-Test p-value,0.245146


White Test (version 1)


,0,1
0,LM Statistic,9.549452
1,LM-Test p-value,0.388174
2,F-Statistic,1.054957
3,F-Test p-value,0.405312


White Test (Version 2)


,0,1
0,LM Statistic,3.447287
1,LM-Test p-value,0.178415
2,F-Statistic,1.732761
3,F-Test p-value,0.182982


## Computer exercises 

### 1 
Run the model for the `sleep75` data set as appears in the book and test if the variance depends on the gender variable. 

In [39]:
sleep75 = woo.dataWoo('sleep75')

regc1 = smf.ols('sleep ~ totwrk + educ + age + I(age**2) + yngkid + male', data=sleep75).fit()
smf.ols('I(regc1.resid**2) ~ male',sleep75).fit().summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     I(regc1.resid ** 2)   R-squared:                       0.002
Model:                             OLS   Adj. R-squared:                  0.000
Method:                  Least Squares   F-statistic:                     1.117
Date:                 Fri, 19 Nov 2021   Prob (F-statistic):              0.291
Time:                         12:20:39   Log-Likelihood:                -10032.
No. Observations:                  706   AIC:                         2.007e+04
Df Residuals:                      704   BIC:                         2.008e+04
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.894e+05   2.05e+04      9.216      0.000    1.49e+05     2.3e+05
male       -2.885e+04   2.73e+04     -1.057      0.291   -8.24e+04    2.47e+04
==============================================================================
Omnibus:                      982.035   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           230588.128
Skew:                           7.369   Prob(JB):                         0.00
Kurtosis:                      90.301   Cond. No.                         2.80
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

> The variance is not statistically different for men than for women. The t statistic is not significant at even 20% 

### 2 
Use the data in `HPRICE1` to obtain the heteroskedasticity-robust standard errors for equation

$$price = \beta_0 + \beta_1*lotsize + \beta_2*sqrft + \beta_3*bdrms $$ 

Discuss any important differences with the usual standard errors. 
(ii)	Repeat part (i) for equation 
$$log(price) = log(lotsize) + log(sqrft) + bdrms$$.
(iii)	What does this example suggest about heteroskedasticity and the transformation used for the dependent variable?

**Take the log-transform of the dependent variable
Taking the log transform of the dependent variable is one of the most commonly used techniques for not only linearizing the dependent variable y but for also 'dampening down' the heteroscedastic variance (if it exists) in y.**


In [41]:
hprice1 = woo.dataWoo('hprice1')
results=smf.ols('price ~ lotsize + sqrft + bdrms', data=hprice1).fit()
# Refined White heteroscedasticity-robust SE:
results_rob=smf.ols('price ~ lotsize + sqrft + bdrms', data=hprice1).fit(cov_type='HC3')
# put all results in stargazer table and 
models = Stargazer([results, results_rob])
#models.covariate_order(['Intercept','female' , 'educ' , 'exper', 'tenure', 'married',])
HTML(models.render_html())

In [42]:
resultslog=smf.ols("np.log(price)~np.log(lotsize)+np.log(sqrft)+bdrms", data=hprice1).fit()
resultslog_rob=smf.ols("np.log(price)~np.log(lotsize)+np.log(sqrft)+bdrms", data=hprice1).fit(cov_type='HC3')

# Refined White heteroscedasticity-robust SE:
# put all results in stargazer table and 
modelslog = Stargazer([resultslog, resultslog_rob])
#models.covariate_order(['Intercept','female' , 'educ' , 'exper', 'tenure', 'married',])
HTML(modelslog.render_html())

&nbsp;
<hr />
<p style="font-family:palatino; text-align: center;font-size: 15px">ECON320 Python Programming Laboratory</a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px">Professor <em> Paloma Lopez de mesa Moyano</em></a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"><span style="color: #6666FF;"><em>paloma.moyano@emory.edu</em></span></p>

<p style="font-family:palatino; text-align: center;font-size: 15px">Department of Economics</a></p>
<p style="font-family:palatino; text-align: center; color: #012169;font-size: 15px">Emory University</a></p>

&nbsp;